<a href="https://colab.research.google.com/github/dorddis/IIIT_PUNE_Hacktoberfest_2022/blob/main/ML_mini_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Objectives:


*  Create new feature from the existing one which we found out from study that it impacts this disease.


*   To know what is the correlation between variables? Can we expect any relationship? Can we expect a value for the person’s blood pressure when we know his/her age?

*   Based on a person’s health profile, can we predict if a person is likely to have CVD?





In [14]:
import pandas as pd
import numpy as np

In [15]:
data=pd.read_csv('cardio_train.csv',sep=';')

In [16]:
data.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [17]:
data.shape

(70000, 13)

from here, we can conclude data has 70000 entries and 13 features.

In [18]:
data.isnull().sum()

id             0
age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
cardio         0
dtype: int64

from this we can conclude, No feature has any missing value.

**Data preparation:**



*  drop duplicated rows if any.
*  drop any unusual feature.


*   Identify Outliers and remove them



*   convert age from days to years.
*   create a column for ranges of age.

*   create a column for body mass index (bmi).
*   create a column for state of blood pressure.














In [19]:
data.duplicated().sum()

0

There is no duplicate entries rows in dataset.

In [20]:
data.drop('id',axis=1,inplace=True)

In [21]:
data.describe()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000
mean,19468.865814,1.349571,164.359229,74.205690,128.817286,96.630414,1.366871,1.226457,0.088129,0.053771,0.803729,0.499700
std,2467.251667,0.476838,8.210126,14.395757,154.011419,188.472530,0.680250,0.572270,0.283484,0.225568,0.397179,0.500003
min,10798.000000,1.000000,55.000000,10.000000,-150.000000,-70.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,17664.000000,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,19703.000000,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,21327.000000,2.000000,170.000000,82.000000,140.000000,90.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,23713.000000,2.000000,250.000000,200.000000,16020.000000,11000.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000


from this we can observe and identify the outliers. like min and max of weight and height deosn't make sense. So, we will remove them as it will wrongly deviate our model.

In [22]:
# Remove weight outliers
weight_min_outlier = data['weight'] > data['weight'].quantile(0.005)
weight_max_outlier = data['weight'] < data['weight'].quantile(0.999)
data = data[(weight_min_outlier) & (weight_max_outlier)]

In [23]:
# Remove height outliers
height_min_outlier = data['height'] > data['height'].quantile(0.005)
height_max_outlier = data['height'] < data['height'].quantile(0.999)
data = data[(height_min_outlier) & (height_max_outlier)]

Now, moving ahead,Blood flows through our body because of a difference in pressure.Our blood pressure is highest at the start of its journey from our heart ( when it enters the aorta )and it is lowest at the end of its journey along progressively smaller branches of arteries.

The systolic pressure (ap_hi) is the higher figure caused by the heart’s contraction, while the diastolic number (ap_lo) is the lower pressure in the arteries.

Therefore, ap_hi should be higher thatn ap_low, and they cannot be negative.Hence, we should remove the negative ones and also, the one where ap_hi is lower.


In [24]:
data[data['ap_hi'] < data['ap_lo']].shape[0]

1217

In 1217 obeservation ap_hi is lower than ap_low, which is incorrect. So,we should remove them.

In [25]:
data = data[data['ap_hi'] > data['ap_lo']].reset_index(drop=True)

In [26]:
def blood_pressure(data):
    if (data['ap_hi'] <= 120) and (data['ap_lo'] <= 80):
        return 'Normal'
    if (data['ap_hi'] >= 120 and data['ap_hi'] < 129) and (data['ap_lo'] < 80):
        return 'Elevated'
    if (data['ap_hi'] >= 129 and data['ap_hi'] <= 179) | (data['ap_lo'] >= 89):
        return 'High_Blood_Pressure'
    if (data['ap_hi'] >= 180) | (data['ap_lo'] >= 120):
        return 'Crisis'

data['blood_pressure'] = data.apply(blood_pressure, axis=1)

here, we created a feature of blood pressure as it is key factor to know in heart related disease.

In [27]:
data['age'] = round(data['age']/365)

since, the age was in days so we converted it into years.

In [28]:
print('Min Age: ', data['age'].min())
print('Max Age: ', data['age'].max())
print('Mean Age: ', data['age'].mean())

data['age_range'] = data['age'].apply(
    lambda x: 'young' if (x >=30 and x <= 40) else ('middle_age' if (x>40 and x<=48) else 'Elderly'))

Min Age:  30.0
Max Age:  65.0
Mean Age:  53.320865880129354


We created new feature of age_range by observing min,max and mean age value.

from studying many papers and articles on this topic, we concluded that BMI feature is essential and hence we calculated it. Also, using standard weight status categories associated with BMI ranges, we created its range new feature.

In [29]:
def BMI(data):
    return data['weight'] / (data['height']/100)**2

data['bmi'] = data.apply(BMI, axis=1)

In [30]:
data['weight_status'] = data['bmi'].apply(
  lambda x :'Underweight' if x <= 18.5 else
   ('Normal' if (x > 18.5 and x <= 24.9) else
    ( 'Overweight' if (x > 24.9 and x <= 29.9)
    else 'obese')) )

In [31]:
data.describe()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,bmi
count,67723.000000,67723.000000,67723.000000,67723.000000,67723.000000,67723.000000,67723.000000,67723.000000,67723.000000,67723.000000,67723.000000,67723.000000,67723.000000
mean,53.320866,1.351077,164.580320,74.244721,128.892902,81.278177,1.364603,1.225979,0.088434,0.053527,0.803479,0.495312,27.447471
std,6.760800,0.477311,7.569775,13.837317,156.320706,9.648790,0.679115,0.571949,0.283927,0.225083,0.397370,0.499982,5.059165
min,30.000000,1.000000,146.000000,45.800000,12.000000,-70.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,14.609204
25%,48.000000,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,23.875115
50%,54.000000,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,26.346494
75%,58.000000,2.000000,170.000000,82.000000,140.000000,90.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,30.116213
max,65.000000,2.000000,189.000000,149.000000,16020.000000,182.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000,58.097313


In [32]:
data['cholesterol'].replace({1: 'normal', 2: 'above normal', 3: 'well above normal'}, inplace = True)

In [34]:
data.head(20)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,blood_pressure,age_range,bmi,weight_status
0,50.0,2,168,62.0,110,80,normal,1,0,0,1,0,Normal,Elderly,21.967120,Normal
1,55.0,1,156,85.0,140,90,well above normal,1,0,0,1,1,High_Blood_Pressure,Elderly,34.927679,obese
2,52.0,1,165,64.0,130,70,well above normal,1,0,0,0,1,High_Blood_Pressure,Elderly,23.507805,Normal
3,48.0,2,169,82.0,150,100,normal,1,0,0,1,1,High_Blood_Pressure,middle_age,28.710479,Overweight
4,48.0,1,156,56.0,100,60,normal,1,0,0,0,0,Normal,middle_age,23.011177,Normal
5,60.0,1,151,67.0,120,80,above normal,2,0,0,0,0,Normal,Elderly,29.384676,Overweight
6,61.0,1,157,93.0,130,80,well above normal,1,0,0,1,0,High_Blood_Pressure,Elderly,37.729725,obese
7,62.0,2,178,95.0,130,90,well above normal,3,0,0,1,1,High_Blood_Pressure,Elderly,29.983588,obese
8,48.0,1,158,71.0,110,70,normal,1,0,0,1,0,Normal,middle_age,28.440955,Overweight
9,54.0,1,164,68.0,110,60,normal,1,0,0,0,0,Normal,Elderly,25.282570,Overweight
